In [1]:
import numpy as np
import torch 
import math

# Compute NLL based on matched results after match_store.py

## Corner-based format with single-variate Gaussian distribution
kl_loss_corner

In [2]:
def compute_nll_corner_single(data_path, ty = 0, info=None):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    nll_list = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = torch.from_numpy(prediction[:,:8])
        target = torch.from_numpy(np.array(tp[i][1]))
        cov = torch.from_numpy(prediction[:,9:])
        std = torch.sqrt(torch.exp(cov))
        if ty == 0:
            std = std
        elif ty == 1:
            std = torch.multiply(std, torch.FloatTensor(info))
        elif ty == 2:
            std = std * info
        predicted_normal_dists = torch.distributions.normal.Normal(pred, std)
        negative_log_prob = - predicted_normal_dists.log_prob(target)
        negative_log_prob = torch.clamp(negative_log_prob, min = 0)
        #print(std)
        #print(negative_log_prob)
        negative_log_prob = torch.sum(negative_log_prob,axis=1)
        #print(negative_log_prob)
        nll_list.extend(negative_log_prob.tolist())
    return sum(nll_list) / len(nll_list) / 8

### disco

In [55]:
data_path = "check/check_loss_two_step_corner/disco/no_rsu/match_all_data_100_5.npy"
compute_nll_corner_single(data_path, 0)

416.2972604126629

In [56]:
# 68.27%
data_path = "check/check_loss_two_step_corner/disco/no_rsu/match_all_data_100_5.npy"
quantile = [5.685563231556715, 5.7333907000666064, 5.8156107496472655, 5.698934531905109, 5.349435233304022, 5.286866601436044, 5.364550922522339, 5.254597136256393]
compute_nll_corner_single(data_path, 1, quantile)

13.494476708883193

In [57]:
# 99.73%
data_path = "check/check_loss_two_step_corner/disco/no_rsu/match_all_data_100_5.npy"
quantile = np.array([48.667429272195506, 47.12830691785341, 47.89193980424878, 42.536190890924544, 42.169949853660526, 46.371733470171094, 41.253330827583945, 41.55867648053199]) / 3
compute_nll_corner_single(data_path, 1, quantile)

1.9925759560542673

In [58]:
data_path = "check/check_loss_two_step_corner/disco/no_rsu/match_all_data_100_5.npy"
compute_nll_corner_single(data_path, 2, 10)

4.084503384071757

### Upperbound

In [59]:
data_path = "check/check_loss_two_step_corner/upperbound/no_rsu/match_all_data_100_5.npy"
compute_nll_corner_single(data_path, 0)

675.9178318133589

In [60]:
# 68.27%
data_path = "check/check_loss_two_step_corner/upperbound/no_rsu/match_all_data_100_5.npy"
quantile = [4.190345185940821, 4.351993435985297, 4.558479310459919, 4.717549387208282, 4.278715029642637, 4.102446480783583, 4.1725601258012475, 3.530299361843318]
compute_nll_corner_single(data_path, 1, quantile)

38.17884247695083

In [61]:
# 99.73%
data_path = "check/check_loss_two_step_corner/upperbound/no_rsu/match_all_data_100_5.npy"
quantile = np.array([49.72913250209128, 68.07916849224074, 54.76645263747507, 76.2931006003976, 46.32874805418221, 76.94147233490142, 50.70841262977031, 77.77131089570797]) / 3
compute_nll_corner_single(data_path, 1, quantile)

2.1025180192343154

In [3]:
data_path = "check/check_loss_two_step_corner/upperbound/no_rsu/match_all_data_100_5.npy"
compute_nll_corner_single(data_path, 2, 10)

6.72424460927033

### Lowerbound

In [62]:
data_path = "check/check_loss_two_step_corner/lowerbound/no_rsu/match_all_data_100_5.npy"
compute_nll_corner_single(data_path, 0)

1006.1331307949978

In [63]:
# 68.27%
data_path = "check/check_loss_two_step_corner/lowerbound/no_rsu/match_all_data_100_5.npy"
quantile = [6.10302951960435, 6.182331226887726, 5.740435199991636, 6.236356538776135, 6.334263986132204, 5.1828751359733305, 6.399557006521807, 5.073939114533967]
compute_nll_corner_single(data_path, 1, quantile)

28.671798591521377

In [64]:
# 99.73%
data_path = "check/check_loss_two_step_corner/lowerbound/no_rsu/match_all_data_100_5.npy"
quantile = np.array([259.92825137969436, 201.07436055964888, 258.2222934053669, 197.05243747520606, 328.5189148613409, 235.0538877505854, 339.6938946129562, 235.97804342045072]) / 3
compute_nll_corner_single(data_path, 1, quantile)

1.5389587415294637

In [4]:
data_path = "check/check_loss_two_step_corner/lowerbound/no_rsu/match_all_data_100_5.npy"
compute_nll_corner_single(data_path, 2, 10)

9.97808175733016

## Center-based format with single-variate Gaussian distribution
kl_loss_center_ind

In [5]:
def corner_to_center_box2d_torch(corners):
    p1 = corners[:,0:2]
    p2 = corners[:,2:4]
    p3 = corners[:,4:6]
    p4 = corners[:,6:8]
    center = (p1+p2+p3+p4)/4
    w = (np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p2-p3, 2), axis=1, keepdims = True)))/2
    h = (np.sqrt(np.sum(np.power(p1-p2, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p3-p4, 2), axis=1, keepdims = True)))/2
    wp = np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True))
    sina = np.divide((p4[:,1:2] - p1[:,1:2]), wp)
    cosa = np.divide((p1[:,0:1] - p4[:,0:1]), wp)
    result = np.concatenate((center, w, h, sina, cosa), axis=1)
    return result

def compute_nll_center_single(data_path, ty = 0, info=None):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    nll_list = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = corner_to_center_box2d_torch(prediction[:,:8])
        target = corner_to_center_box2d_torch(np.array(tp[i][1]))
        pred = torch.from_numpy(pred)
        target = torch.from_numpy(target)
        cov = torch.from_numpy(prediction[:,9:])
        std = torch.sqrt(torch.exp(cov))
        if ty == 0:
            std = std
        elif ty == 1:
            std = torch.multiply(std, torch.FloatTensor(info))
        elif ty == 2:
            std = std * info
        predicted_normal_dists = torch.distributions.normal.Normal(pred, std)
        negative_log_prob = - predicted_normal_dists.log_prob(target)
        negative_log_prob = torch.clamp(negative_log_prob, min=0)
        negative_log_prob = torch.sum(negative_log_prob,axis=1)
        nll_list.extend(negative_log_prob.tolist())
    return sum(nll_list) / len(nll_list) / 6

### disco

In [66]:
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
compute_nll_center_single(data_path, 0)

389.0953396145462

In [67]:
# 68.27%
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
quantile = [5.468984395243384, 4.89869516767041, 16.09847077479864, 3.0882512529684347, 1.0243907317028713, 4.811753752332904]
compute_nll_center_single(data_path, 1, quantile)

214.13620986247267

In [68]:
# 99.73%
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
quantile = np.array([48.40839894811819, 25.026924203602636, 131.90336169825727, 15.255118427560566, 59.67641819394204, 71.02182939111708]) / 3
compute_nll_center_single(data_path, 1, quantile)

0.853390490777656

In [69]:
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
compute_nll_center_single(data_path, 2, 10)

3.6698438872753427

### upperbound

In [70]:
data_path = "check/check_loss_two_step_center_ind/upperbound/no_rsu/match_all_data_100_5.npy"
compute_nll_center_single(data_path, 0)

982.4924730324802

In [71]:
# 68.27%
data_path = "check/check_loss_two_step_center_ind/upperbound/no_rsu/match_all_data_100_5.npy"
quantile = [3.9270005592398936, 3.7293620083399235, 12.661970846621758, 3.507125215592104, 1.3240707560700242, 3.309770337192223]
compute_nll_center_single(data_path, 1, quantile)

362.74823739559133

In [72]:
# 99.73%
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
quantile = np.array([31.340185602346363, 24.276376908593676, 118.64887029003187, 18.253235606033712, 109.68835650542137, 56.51205738298187]) / 3
compute_nll_center_single(data_path, 1, quantile)

0.5777378239962699

In [6]:
data_path = "check/check_loss_two_step_center_ind/upperbound/no_rsu/match_all_data_100_5.npy"
compute_nll_center_single(data_path, 2, 10)

9.627653719321062

### lowerbound

In [73]:
data_path = "check/check_loss_two_step_center_ind/lowerbound/no_rsu/match_all_data_100_5.npy"
compute_nll_center_single(data_path, 0)

3440.4599480614484

In [74]:
# 68.27%
data_path = "check/check_loss_two_step_center_ind/lowerbound/no_rsu/match_all_data_100_5.npy"
quantile = [4.824761883365095, 4.435752094000069, 14.379824436595687, 2.8108565130701026, 1.0106651376400175, 4.404153510782447]
compute_nll_center_single(data_path, 1, quantile)

736.2581263012539

In [75]:
# 99.73%
data_path = "check/check_loss_two_step_center_ind/disco/no_rsu/match_all_data_100_5.npy"
quantile = np.array([44.646381707137465, 28.842831596825288, 127.50113292712857, 18.538402799100826, 81.66889262783303, 53.23585004452376]) / 3
compute_nll_center_single(data_path, 1, quantile)

0.7287420792220193

In [7]:
data_path = "check/check_loss_two_step_center_ind/lowerbound/no_rsu/match_all_data_100_5.npy"
compute_nll_center_single(data_path, 2, 10)

34.180653841945905